# Exploration of execution

Execution 

```{bash}
cd /home/ec2-user/SageMaker/foodi-ml
source activate python3
export DATA_PATH=/home/ec2-user/SageMaker/data/
python run.py options/adapt/foodi-ml/i2t.yaml

#nvidia-smi
```

In [1]:
import os
os.chdir("/home/ec2-user/SageMaker/foodi-ml/")

In [2]:
#!pip install https://download.pytorch.org/whl/cu100/torch-1.1.0-cp36-cp36m-linux_x86_64.whl

In [4]:
#!pip install -r requirements.txt

In [5]:
import os
import torch
from tqdm import tqdm

import params
from retrieval.train import train
from retrieval.utils import helper
from retrieval.model import loss
from retrieval.model.model import Retrieval
from retrieval.data.loaders import get_loaders
from retrieval.utils.logger import create_logger
from retrieval.utils.helper import load_model
from retrieval.utils.file_utils import load_yaml_opts, parse_loader_name

In [7]:
from addict import Dict

# Functions

In [9]:
def get_data_path(opt):
    if 'DATA_PATH' not in os.environ:
        if not opt.dataset.data_path:
            raise Exception('''
                DATA_PATH not specified.
                Please, run "$ export DATA_PATH=/path/to/dataset"
                or add path to yaml file
            ''')
        return opt.dataset.data_path
    else:
        return os.environ['DATA_PATH']

In [10]:
def get_tokenizers(train_loader):
    tokenizers = train_loader.dataset.tokenizer
    if type(tokenizers) != list:
        tokenizers = [tokenizers]
    return tokenizers

In [11]:
def set_criterion(opt, model):
    if 'name' in opt.criterion:
        logger.info(opt.criterion)
        multimodal_criterion = loss.get_loss(**opt.criterion)
        multilanguage_criterion = loss.get_loss(**opt.criterion)
    else:
        multimodal_criterion = loss.ContrastiveLoss(**opt.criterion)
        multilanguage_criterion = loss.ContrastiveLoss(**opt.ml_criterion)
    set_model_criterion(opt, model, multilanguage_criterion, multimodal_criterion)
    # return multimodal_criterion, multilanguage_criterion


def set_model_criterion(opt, model, multilanguage_criterion, multimodal_criterion):
    model.mm_criterion = multimodal_criterion
    model.ml_criterion = None
    if len(opt.dataset.adapt.data) > 0:
        model.ml_criterion = multilanguage_criterion

# Run

In [12]:
os.environ["DATA_PATH"] = "/home/ec2-user/SageMaker/data/"

In [13]:
options = "options/adapt/foodi-ml/i2t.yaml"

In [14]:
args = {
    "options": options,
}
args = Dict(args)
opt = load_yaml_opts(args.options)

In [15]:
logger = create_logger(level='debug' if opt.engine.debug else 'info')
#logger.info(f'Used args   : \n{args}')
#logger.info(f'Used options: \n{opt}')

In [16]:
# Get path of the data
data_path = get_data_path(opt)

In [17]:
# Get loaders
train_loader, val_loaders, adapt_loaders = get_loaders(data_path, args.local_rank, opt)

2021-08-17 13:32:05,903 - [INFO    ] - Loaded vocab containing 2487 tokens
2021-08-17 13:32:05,904 - [INFO    ] - Loaded from .vocab_cache/foodiml_vocab.json.
2021-08-17 13:32:05,904 - [INFO    ] - Created tokenizer with init 2487 tokens.
2021-08-17 13:32:05,947 - [INFO    ] - [FoodiML] Loaded 8011 images and 8011 annotations.
2021-08-17 13:32:05,951 - [INFO    ] - Loaded vocab containing 2487 tokens
2021-08-17 13:32:05,952 - [INFO    ] - Loaded from .vocab_cache/foodiml_vocab.json.
2021-08-17 13:32:05,952 - [INFO    ] - Created tokenizer with init 2487 tokens.
2021-08-17 13:32:05,987 - [INFO    ] - [FoodiML] Loaded 0 images and 0 annotations.
2021-08-17 13:32:05,988 - [INFO    ] - Adapt loaders: 0


In [18]:
tokenizers = get_tokenizers(train_loader)

In [19]:
model = Retrieval(**opt.model, tokenizers=tokenizers)

2021-08-17 13:32:08,684 - [INFO    ] - Image encoder created: ('full_image',)
2021-08-17 13:32:08,973 - [INFO    ] - Text encoder created: gru_glove
2021-08-17 13:32:09,043 - [INFO    ] - Created similarity: AdaptiveEmbeddingI2T(
  (norm): Normalization(
    (norm): BatchNorm1d(2048, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
  )
  (adapt_txt): ADAPT(
    (fc_gamma): Sequential(
      (0): Linear(in_features=2048, out_features=2048, bias=True)
    )
    (fc_beta): Sequential(
      (0): Linear(in_features=2048, out_features=2048, bias=True)
    )
  )
  (fovea): Fovea(smooth=10,train_smooth: False)
)
2021-08-17 13:32:09,190 - [INFO    ] - Setting devices: img: cuda,txt: cuda, loss: cuda
2021-08-17 13:32:09,190 - [INFO    ] - Using similarity: ('adapt_i2t',)


In [20]:
print_fn = (lambda x: x) if not model.master else tqdm.write

In [21]:
set_criterion(opt, model)

In [22]:
# Trainer
trainer = train.Trainer(
        model=model,
        args=opt,
        sysoutlog=print_fn,
        path=opt.exp.outpath,
        world_size=1 # TODO
)

In [23]:
trainer.setup_optim(
        lr=opt.optimizer.lr,
        lr_scheduler=opt.optimizer.lr_scheduler,
        clip_grad=opt.optimizer.grad_clip,
        log_grad_norm=False,
        log_histograms=False,
        optimizer=opt.optimizer,
        freeze_modules=opt.model.freeze_modules
)

2021-08-17 13:32:12,691 - [INFO    ] - lr 0.001
2021-08-17 13:32:12,692 - [INFO    ] - [0.5, 2.0, 4000]
2021-08-17 13:32:12,692 - [INFO    ] - [10000, 20000, 3000]


Freezing model.txt_enc.embed.glove
lr: 0.001, #layers: 478, #params: 99,845,812
Total Params: 102,349,912, 


In [25]:
trainer.fit(
        train_loader=train_loader,
        valid_loaders=val_loaders,
        lang_loaders=adapt_loaders,
        nb_epochs=opt.engine.nb_epochs,
        valid_interval=opt.engine.valid_interval,
        log_interval=opt.engine.print_freq
)

logs/foodi-ml/adapt_i2t/ already exists! Do you want to rewrite it? [y/n]  y


Steps :   1%|          | 8/802 [00:03<05:56,  2.23it/s]
                                              2.23it/s]
Steps :   1%|          | 10/802 [00:04<05:53,  2.24it/s]

Batch_Time:   0.407761, Countdown:  50.000000, Epoch:   0.000000, Iteration:  10.000000, K:   0.086441, Loss:  33.452599, Lr_Base:   0.000504, Norm: 117.124039, Total_Loss:  33.452599



Steps :   2%|▏         | 18/802 [00:07<05:41,  2.30it/s]
                                               2.31it/s]
Steps :   2%|▏         | 20/802 [00:08<05:38,  2.31it/s]

Batch_Time:   0.396405, Countdown:  50.000000, Epoch:   0.000000, Iteration:  20.000000, K:   0.165410, Loss:  30.421429, Lr_Base:   0.000508, Norm:  94.119436, Total_Loss:  30.421429


...

Batch_Time:   0.447006, Countdown:  50.000000, Epoch:   0.000000, Iteration:  70.000000, K:   0.468925, Loss:  20.099714, Lr_Base:   0.000526, Norm:  68.479503, Total_Loss:  20.099714



Steps :  10%|▉         | 78/802 [00:33<05:09,  2.34it/s]
                                               2.34it/s]
Steps :  10%|▉         | 80/802 [00:34<05:08,  2.34it/s]

Batch_Time:   0.416992, Countdown:  50.000000, Epoch:   0.000000, Iteration:  80.000000, K:   0.514832, Loss:  19.593702, Lr_Base:   0.000530, Norm:  66.754051, Total_Loss:  19.593702



Steps :  10%|█         | 82/802 [00:35<05:07,  2.34it/s]


KeyboardInterrupt: 

In [26]:
torch.__version__

'1.1.0'

In [ ]:
#!pip install https://download.pytorch.org/whl/cu100/torch-1.0.1-cp36-cp36m-win_amd64.whl

In [8]:
import torch
a = torch.randn(3, 3, device="cuda:0")
a = torch.mm(a, a.t()) # make symmetric positive-definite
torch.cholesky(a)

tensor([[ 1.5002,  0.0000,  0.0000],
        [-0.5008,  1.0633,  0.0000],
        [ 0.0642,  0.4374,  1.6847]], device='cuda:0')